In [1]:
from z import *
from trainer import *
from torchvision import datasets, transforms
from torchsummary import summary

In [2]:
train_dataset = datasets.MNIST('./MNIST', train=True, download=True,
                               transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = datasets.MNIST('./MNIST', train=False, 
                              transform=transforms.Compose([transforms.ToTensor()]))

In [3]:
model = nn.Sequential(
    ZConv2d(1, 16, 4, 5),
    SoftPool2d(16, 2, 2),
    ZConv2d(16, 32, 4, 5),
    SoftPool2d(32, 2, 2),
    nn.Flatten(),
    nn.Dropout(0.5),
    ZLinear(32 * 4 * 4, 10, 4)
).to(0)

In [4]:
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 24, 24]           1,664
           ZConv2d-2           [-1, 16, 24, 24]               0
        SoftPool2d-3           [-1, 16, 12, 12]               0
            Conv2d-4            [-1, 128, 8, 8]          51,328
           ZConv2d-5             [-1, 32, 8, 8]               0
        SoftPool2d-6             [-1, 32, 4, 4]               0
           Flatten-7                  [-1, 512]               0
           Dropout-8                  [-1, 512]               0
            Linear-9                   [-1, 40]          20,520
          ZLinear-10                   [-1, 10]               0
Total params: 73,512
Trainable params: 73,512
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.46
Params size (MB): 0.28
Estimated Tot

In [5]:
def loss_func(model, batch, scope):
  x, y = batch
  _y = model(x)
  loss = F.cross_entropy(_y, y)
  scope["metrics"]["Accuracy"] = float(torch.sum((torch.argmax(_y, dim=1) == y).type(torch.float))) / len(scope["dataset"])
  return loss, _y

In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [7]:
acc = 0
def on_epoch(scope):
  global acc
  _acc = scope["val_metrics"]["Accuracy"]
  if _acc > acc:
    acc = _acc
    torch.save(model.state_dict(), "mnist.model")
    print("Model saved!")

In [8]:
train(model, loss_func, train_dataset, test_dataset, optimizer, device=0, epochs=100, batch_size=256, on_val_epoch=on_epoch)

  1%|          | 2/235 [00:00<00:12, 18.81it/s]

Epoch #1


 10%|█         | 4/40 [00:00<00:00, 36.73it/s]

	Train Loss = 0.5257179239543818
	Train Accuracy = 0.8483333333333347


  1%|▏         | 3/235 [00:00<00:09, 25.55it/s]

...

 10%|█         | 4/40 [00:00<00:00, 38.95it/s]

	Train Loss = 0.00587170996978954
	Train Accuracy = 0.9979499999999958


100%|██████████| 40/40 [00:00<00:00, 43.65it/s]

	Validation Loss = 0.018194401331021213
	Validation Accuracy = 0.9956999999999993


Sequential(
  (0): ZConv2d(
    (layer): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
  )
  (1): SoftPool2d()
  (2): ZConv2d(
    (layer): Conv2d(16, 128, kernel_size=(5, 5), stride=(1, 1))
  )
  (3): SoftPool2d()
  (4): Flatten(start_dim=1, end_dim=-1)
  (5): Dropout(p=0.5, inplace=False)
  (6): ZLinear(
    (layer): Linear(in_features=512, out_features=40, bias=True)
  )
)

In [9]:
print("Best Accuracy = " + str(acc))

Best Accuracy = 0.9962999999999993
